In [1]:
#Importing Dependencies
import pandas as pd
import json
import datetime
import numpy as np
from sqlalchemy import create_engine


In [2]:
#Reading state_vaccine csv file
state_vaccine_csv_file = "Source 1/us_state_vaccinations.csv"
state_vaccine_data_df = pd.read_csv(state_vaccine_csv_file)
state_vaccine_data_df.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN


In [3]:
#Reading cdc_data csv file
cdc_data_df_csv_file = "Source 2/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"
cdc_data_df = pd.read_csv(cdc_data_df_csv_file)
cdc_data_df.head()



,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,05/04/2020,KY,5245,5245.0,0.0,115,0.0,261,261.0,0.0,8,0.0,05/05/2020 05:25:08 PM,Agree,Agree
1,03/23/2020,NH,101,NaN,NaN,23,NaN,1,NaN,NaN,1,NaN,03/26/2020 04:22:39 PM,Not agree,Not agree
2,08/21/2020,MD,103523,NaN,NaN,624,0.0,3685,3546.0,139.0,11,1.0,08/22/2020 02:21:09 PM,NaN,Agree
3,02/27/2021,MS,294795,182857.0,111938.0,704,478.0,6681,4672.0,2009.0,12,7.0,02/28/2021 02:48:18 PM,Agree,Agree
4,03/12/2020,NJ,29,NaN,NaN,6,NaN,1,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree


In [16]:
#Reading state_lookup csv file
lookup_data_df_csv_file = "Source 1/state_province_lookup.csv"
lookup_data_df = pd.read_csv(lookup_data_df_csv_file)
lookup_data_df.head()


,State,Code
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [4]:
#counting rows for data set
state_vaccine_data_df.count()


date                                   6913
location                               6913
total_vaccinations                     6643
total_distributed                      6531
people_vaccinated                      6519
people_fully_vaccinated_per_hundred    6101
total_vaccinations_per_hundred         6203
people_fully_vaccinated                6437
people_vaccinated_per_hundred          6178
distributed_per_hundred                6190
daily_vaccinations_raw                 6848
daily_vaccinations                     6848
daily_vaccinations_per_million         6393
share_doses_used                       6531
dtype: int64

In [5]:
#counting rows for data set
cdc_data_df.count()

submission_date    27660
state              27660
tot_cases          27660
conf_cases         12454
prob_cases         12453
new_case           27660
pnew_case          22749
tot_death          27660
conf_death         12818
prob_death         12818
new_death          27660
pnew_death         22558
created_at         27660
consent_cases      22589
consent_deaths     23511
dtype: int64

In [6]:
cdc_data_df.head()


,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,05/04/2020,KY,5245,5245.0,0.0,115,0.0,261,261.0,0.0,8,0.0,05/05/2020 05:25:08 PM,Agree,Agree
1,03/23/2020,NH,101,NaN,NaN,23,NaN,1,NaN,NaN,1,NaN,03/26/2020 04:22:39 PM,Not agree,Not agree
2,08/21/2020,MD,103523,NaN,NaN,624,0.0,3685,3546.0,139.0,11,1.0,08/22/2020 02:21:09 PM,NaN,Agree
3,02/27/2021,MS,294795,182857.0,111938.0,704,478.0,6681,4672.0,2009.0,12,7.0,02/28/2021 02:48:18 PM,Agree,Agree
4,03/12/2020,NJ,29,NaN,NaN,6,NaN,1,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree


In [7]:
#filtering based on most current date from file
cdc_dataGB_df = cdc_data_df[cdc_data_df["submission_date"] == "04/26/2021"]
cdc_dataGB_df.head(2)

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
135,04/26/2021,OR,181321,181321.0,0.0,621,0.0,2486,2486.0,0.0,1,0.0,04/27/2021 01:56:18 PM,Agree,Agree
383,04/26/2021,VI,3090,NaN,NaN,4,0.0,27,NaN,NaN,0,0.0,04/27/2021 01:56:18 PM,NaN,NaN


In [8]:
#dropping unnecessary columns
cdc_dataGB_df.drop(columns = ["created_at","consent_cases","consent_deaths"], axis = 1, inplace = True)
cdc_dataGB_df.head()

C:\Users\John-Doe\anaconda3\envs\pydata\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death
135,04/26/2021,OR,181321,181321.0,0.0,621,0.0,2486,2486.0,0.0,1,0.0
383,04/26/2021,VI,3090,NaN,NaN,4,0.0,27,NaN,NaN,0,0.0
1406,04/26/2021,DE,103459,96584.0,6875.0,237,21.0,1617,1476.0,141.0,1,0.0
2230,04/26/2021,NV,313680,NaN,NaN,280,0.0,5433,NaN,NaN,11,0.0
2302,04/26/2021,NE,219090,NaN,NaN,258,0.0,2242,NaN,NaN,0,0.0


In [9]:
#converting NaN and null values to 0
cdc_dataGB_df.fillna(value = 0, inplace = True)
cdc_dataGB_df.head()

C:\Users\John-Doe\anaconda3\envs\pydata\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death
135,04/26/2021,OR,181321,181321.0,0.0,621,0.0,2486,2486.0,0.0,1,0.0
383,04/26/2021,VI,3090,0.0,0.0,4,0.0,27,0.0,0.0,0,0.0
1406,04/26/2021,DE,103459,96584.0,6875.0,237,21.0,1617,1476.0,141.0,1,0.0
2230,04/26/2021,NV,313680,0.0,0.0,280,0.0,5433,0.0,0.0,11,0.0
2302,04/26/2021,NE,219090,0.0,0.0,258,0.0,2242,0.0,0.0,0,0.0


In [10]:
#filtering based on most current date from file
state_vaccine_data_df = state_vaccine_data_df[state_vaccine_data_df["date"] == "2021-04-26"]
state_vaccine_data_df.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
104,2021-04-26,Alabama,2478227.0,3960530.0,1549782.0,21.75,50.54,1066659.0,31.61,80.77,6818.0,25809.0,5264.0,0.626
210,2021-04-26,Alaska,540565.0,759125.0,300706.0,34.04,73.89,249036.0,41.11,103.77,935.0,3292.0,4500.0,0.712
316,2021-04-26,American Samoa,37590.0,47010.0,21165.0,29.73,67.50,16557.0,38.01,84.42,0.0,231.0,4148.0,0.800
422,2021-04-26,Arizona,4912269.0,6285415.0,2900707.0,27.61,67.49,2009937.0,39.85,86.35,39748.0,51720.0,7106.0,0.782
528,2021-04-26,Arkansas,1734405.0,2586420.0,1053861.0,24.22,57.47,730955.0,34.92,85.71,9654.0,16576.0,5493.0,0.671


In [11]:
#dropping unnecessary columns
state_vaccine_data_df.drop(columns = ["share_doses_used","daily_vaccinations_per_million","daily_vaccinations","daily_vaccinations_raw","distributed_per_hundred","people_vaccinated_per_hundred","total_vaccinations_per_hundred","people_fully_vaccinated_per_hundred"], axis = 1, inplace = True)
state_vaccine_data_df.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated
104,2021-04-26,Alabama,2478227.0,3960530.0,1549782.0,1066659.0
210,2021-04-26,Alaska,540565.0,759125.0,300706.0,249036.0
316,2021-04-26,American Samoa,37590.0,47010.0,21165.0,16557.0
422,2021-04-26,Arizona,4912269.0,6285415.0,2900707.0,2009937.0
528,2021-04-26,Arkansas,1734405.0,2586420.0,1053861.0,730955.0


In [12]:
rds_connection_string = "postgres:Qwerty1234@localhost:5432/ETL_project_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
engine.table_names()

['state_lookup', 'vaccine_data', 'cdc_data']

In [14]:
state_vaccine_data_df.to_sql(name='vaccine_data', con=engine, if_exists='append', index=False)

In [15]:
cdc_dataGB_df.to_sql(name='cdc_data', con=engine, if_exists='append', index=False)

In [18]:
lookup_data_df.to_sql(name='state_lookup', con=engine, if_exists='append', index=False)